<ul>
    <li> In this project, 12 EMG sensors were used. 9 features were extracted from each sensor leading to a 108 features set and there are 8 outputs</li>
    <li> Features and outputs were divided into left side and right side.</li>
    <li> Number of input EMG Featurs is 54 features and each side will have 4 target values (2 angles and 2 torques)</li>
    <li> In some models I will add the angles as an input, and try to predict the moment only</li>
</ul>

## Setups

In [1]:
from Custom.WindowGenerator import WindowGenerator
import matplotlib as mpl
from functools import partial
from sklearn import metrics
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
from tensorflow import keras
K = keras.backend

mpl.rcParams['figure.dpi'] = 110
Weight = {"S02": 60.5, "S02": 67.8}
model_dic = {}

if not tf.test.is_built_with_cuda():
    raise print("No GPU found")

In [17]:
subject = "02"
w = Weight[f'S{subject}']
dataset_folder = f"../Dataset/S{subject}/"
trials = [f"S{subject}_train_01", f"S{subject}_train_02", f"S{subject}_val", f"S{subject}_test"]
trials = list(map(lambda x: f"{dataset_folder}{x}_dataset.csv", trials))

## Scaling

In [18]:
def scale_features(data, mean, std):
    """
    standard deviation the input (mean=0, std=1)
    """
    data.iloc[:,:-8] = (data.iloc[:,:-8]-mean)/std
    return data

def scale_moment(data, weight, scale=False):
    if scale:
        data.iloc[:,-4:] = data.iloc[:,-4:]/weight
    else:
        pass
    return data

def scale_dataset(train_01_df, train_02_df, val_df, test_df, weight, MinMax=True):
    if not MinMax:
        train_mean = train_01_df.iloc[:,:-8].mean()
        train_std = train_01_df.iloc[:,:-8].std()
        for data in [train_01_df, train_02_df, val_df, test_df]:
            data = scale_features(data, train_mean, train_std)
            data = scale_moment(data,weight)
    else:
        scaler = MinMaxScaler((0,1))
        scaler.fit(train_01_df.iloc[:,:-8])
        for data in [train_01_df, train_02_df, val_df, test_df]:
            data.iloc[:,:-8] = scaler.transform(data.iloc[:,:-8])
            data = scale_moment(data,weight)
    return train_01_df, train_02_df, val_df, test_df

### Import and scale the data

In [19]:
train_01_df = pd.read_csv(trials[0], index_col='time')
train_02_df = pd.read_csv(trials[1], index_col='time')
val_df = pd.read_csv(trials[2], index_col='time')
test_df = pd.read_csv(trials[3], index_col='time')
# train_01_df.describe().transpose()

## Scaling the data
train_01_df, train_02_df, val_df, test_df = scale_dataset(train_01_df, train_02_df, val_df, test_df, w, MinMax=False)
# train_01_df.describe().transpose()

## Window Generator

## Train/Evaluation function

In [20]:
def make_dataset(window_object):
    train_set = window_object.get_train_dataset()
    val_set = window_object.get_val_dataset()
    test_set = window_object.get_evaluation_set()
    return train_set, val_set, test_set

### Models

#### Custom loss function

In [21]:
def custom_loss(y_true, y_pred):
    error = tf.reshape(y_true - y_pred,(-1,1))
    error = error[~tf.math.is_nan(error)]
    return tf.reduce_mean(tf.square(error), axis=0)

#### Custom evaluation functions

In [22]:
def nan_R2(y_true, y_pred):
    R2 = []
    _, l = np.shape(y_true)
    for i in range(l):
        y_true_col = y_true[:,i]
        y_pred_col = y_pred[:,i]
        logic = np.isfinite(y_true_col)
        y_true_col = y_true_col[logic]
        y_pred_col = y_pred_col[logic]
        R2.append(metrics.r2_score(y_true_col, y_pred_col))
    return np.around(R2,4)

def nan_rmse(y_true, y_pred):
    MSE = partial(metrics.mean_squared_error, squared=False)
    rmse=[]
    _, l = np.shape(y_true)
    for i in range(l):
        y_true_col = y_true[:,i]
        y_pred_col = y_pred[:,i]
        logic = np.isfinite(y_true_col)
        y_true_col = y_true_col[logic]
        y_pred_col = y_pred_col[logic]
        rmse.append(MSE(y_true_col, y_pred_col))
    return np.around(rmse,2)

#### LSTM model

In [23]:
def create_lstm_model(window_object, lr=0.001):
    keras.backend.clear_session()
    custom_LSTM = partial(layers.LSTM, dropout=0.3) #kernel_regularizer='l2', recurrent_regularizer='l2', activity_regularizer='l2')
    lstm_model = keras.models.Sequential([
        custom_LSTM(32, return_sequences=True,
                    input_shape=(window_object.input_width, window_object.features_num)),
        custom_LSTM(32, return_sequences=True),
        custom_LSTM(32, return_sequences=True),
        layers.Dense(window_object.out_nums)
    ])
    lstm_model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), loss=custom_loss)
    return lstm_model

model_dic["lstm_model"] = create_lstm_model

#### CNN model

In [24]:
def create_conv_model(window_object, lr=0.001):
    keras.backend.clear_session()
    conv_model = keras.models.Sequential([
        layers.BatchNormalization(input_shape=(window_object.input_width, window_object.features_num)),
        layers.Conv1D(filters=10, kernel_size=3, strides=1, padding='same'),
        layers.BatchNormalization(),
        layers.Conv1D(filters=20, kernel_size=3, strides=1, padding='same'),
        layers.Conv1D(filters=window_object.out_nums, kernel_size=1, strides=1)
    ])
    conv_model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), loss="MSE")
#     conv_model.summary()
    return conv_model
model_dic["conv_model"] = create_conv_model

## Model training function

In [25]:
def train_fit(window_object, model_name, epochs=1, lr=0.001, eval_only=False, load_best=False):
    # setup model folder
    global folder
    folder = f"../Results/indiviuals/{model_name}/S{subject}/"
    if not os.path.exists(folder):
        os.makedirs(folder)
        
    # Get all dataset
    train_set, val_set, test_set = make_dataset(window_object)
    ##############################################################################################################
    # Load and compile new model
    model = model_dic[model_name](window_object, lr)
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath = f"{folder}S{subject}_{model_name}.hdf5",
                                save_weights_only = True, monitor = 'val_loss',
                                save_best_only = True)
    if load_best:
        try:
            model.load_weights(f"{folder}/S{subject}_{model_name}.hdf5")
        except:
            print("No saved model existing. weights will be initialized")
    ##############################################################################################################
    #### Train Model
    try:
        if not eval_only:
            history = model.fit(x=train_set, validation_data=val_set,
                                epochs=epochs, callbacks=[model_checkpoint_callback])
            plot_learning_curve(history)
            # Load the best model
            model.load_weights(f"{folder}/S{subject}_{model_name}.hdf5")
        else:
            history = "No training was conducted"
            model.load_weights(f"{folder}/S{subject}_{model_name}.hdf5")
    except KeyboardInterrupt:
        history = "\n\nTrains stopped manually"
        print(history)
    except OSError: # If no saved model to be evaluated exist
        print("No saved model existing. weights will be initialized")
    ##############################################################################################################
    ## Get predictions and real values
    y_pred = model.predict(test_set)
    if len(y_pred.shape)==3:
        y_pred = y_pred[:,-1,:]
    ## Get real outputs
    for _,y_true in test_set.as_numpy_iterator():
        break
    y_true = y_true[:,-1,:]
    ##############################################################################################################
    ## Evaluation and plot
    r2_score = nan_R2(y_true, y_pred)
    rmse_result = nan_rmse(y_true, y_pred)
    plot_results(y_true, y_pred, r2_score, rmse_result)
    
    return history, y_true, y_pred, r2_score, rmse_result

In [26]:
def plot_learning_curve(history):
    if history==None:
        print("No train history was found")
        return None
    else:
        plt.plot(history.epoch, history.history['loss'], history.epoch, history.history['val_loss'])
        plt.show()
        plt.savefig(f"{folder}learning_curve.png")
        plt.close()
        return None

In [27]:
def plot_results(y_true, y_pred, R2_score, rmse_result):
    global time
    time = [i/20 for i in range(len(y_true))]
    labels = ["Knee moment", "Ankle moment"]
    for i, col in enumerate(labels):
        plt.subplot(2,1,i+1)
        print(f"{col} R2 score: {R2_score[i]}")
        print(f"{col} RMSE result: {rmse_result[i]}")
        plt.plot(time, y_true[:, i],
                 time, y_pred[:, i])
        plt.title(col)
#         plt.legend(["y_true", "y_pred"])
        plt.xlim((time[-600],time[-100]))
        if "moment" in col:
            plt.xlabel("Time [s]")
        if i+1 == 1:
            plt.ylabel("Angle [Degree]")
        elif i+1==3:
            plt.ylabel("Moment [Nm]")
    # plt.savefig(f"{folder}{labels[col]}.png")
    plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)
    plt.show()
    plt.close()

## Main

In [28]:
model_name = "lstm_model"

In [29]:
w1 = WindowGenerator(train_01_df=train_01_df, train_02_df=train_02_df,
                     val_df=val_df, test_df=test_df, input_width=20, out_nums=2)
                     
history, y_true, y_pred, r2, rmse = train_fit(w1, model_name, epochs=100, eval_only=False, load_best=False)

Epoch 1/100


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  Filter predicate `f` must return a scalar bool.
	 [[node IteratorGetNext (defined at Users\amged\AppData\Local\Temp/ipykernel_16532/1246502751.py:26) ]]
	 [[IteratorGetNext/_2]]
  (1) Invalid argument:  Filter predicate `f` must return a scalar bool.
	 [[node IteratorGetNext (defined at Users\amged\AppData\Local\Temp/ipykernel_16532/1246502751.py:26) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_17662]

Function call stack:
train_function -> train_function
